In [1]:
import sys
sys.path.append("..")

In [2]:
import os

In [3]:
import torch
import torch.optim as optim

In [4]:
import train_semantic_segmentation as semseg
from train_semantic_segmentation import (
    call_many,
    save_model_on_better_miou,
    save_segmentations_for_image,
    save_interesting_images
)

In [5]:
from matplotlib import pyplot as plt

In [6]:
EXPERIMENT_NAME = 'experiments/overfit-no-chandrop-small'

In [7]:
DATA_PATHS = {
    'source': os.path.join('../../data/VOCdevkit/VOC2012/JPEGImages'),
    'segmentation': os.path.join('../../data/VOCdevkit/VOC2012/SegmentationClass'),
    'train': os.path.join('../../data/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt'),
    'val': os.path.join('../../data/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt')
}

In [8]:
train_loader, val_loader, test_loader = semseg.load_data(
    DATA_PATHS['source'],
    DATA_PATHS['segmentation'],
    DATA_PATHS['train'],
    DATA_PATHS['val'],
    DATA_PATHS['val']
)

In [9]:
val_loader_with_viewable_transforms = val_loader.dataset.with_viewable_transforms()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
learning_rate = 0.007
epochs = 50

In [ ]:
model = semseg.DeepLabModel(input_channels=3, num_classes=21).to(device)

conv1.weight -> low_level_net.conv1.weight
bn1.weight -> low_level_net.bn1.weight
bn1.bias -> low_level_net.bn1.bias
bn1.running_mean -> low_level_net.bn1.running_mean
bn1.running_var -> low_level_net.bn1.running_var
layer1.0.conv1.weight -> low_level_net.low_level_block.net.0.net.conv1.weight
layer1.0.bn1.weight -> low_level_net.low_level_block.net.0.net.bn1.weight
layer1.0.bn1.bias -> low_level_net.low_level_block.net.0.net.bn1.bias
layer1.0.bn1.running_mean -> low_level_net.low_level_block.net.0.net.bn1.running_mean
layer1.0.bn1.running_var -> low_level_net.low_level_block.net.0.net.bn1.running_var
layer1.0.conv2.weight -> low_level_net.low_level_block.net.0.net.conv2.weight
layer1.0.bn2.weight -> low_level_net.low_level_block.net.0.net.bn2.weight
layer1.0.bn2.bias -> low_level_net.low_level_block.net.0.net.bn2.bias
layer1.0.bn2.running_mean -> low_level_net.low_level_block.net.0.net.bn2.running_mean
layer1.0.bn2.running_var -> low_level_net.low_level_block.net.0.net.bn2.running_var

In [ ]:
criterion = semseg.segmentation_cross_entropy_loss(size_average=None,
                                                   ignore_index=255,
                                                   device=device)
optimizer = optim.SGD(semseg.differential_learning_rates(model, [
                          ((model.feature_detection_layers, ), 1),
                          ((model.spatial_pyramid_pooling, model.decoder), 10)
                      ], learning_rate),
                      momentum=0.9,
                      weight_decay=5e-4,
                      nesterov=False)
scheduler = semseg.PolynomialLearningRateScheduler(optimizer,
                                                   learning_rate,
                                                   epochs,
                                                   len(train_loader))

In [ ]:
def log_statistics_to_notebook(stats):
    print(stats)

In [ ]:
semseg.training_loop(model,
                     train_loader,
                     val_loader,
                     criterion,
                     optimizer,
                     scheduler,
                     device,
                     epochs=epochs,
                     statistics_callback=call_many(
                          semseg.log_statistics(os.path.join(EXPERIMENT_NAME, 'logs', 'statistics'), False),
                          # Take the first image from the first three batches
                          *[save_segmentations_for_image(model,
                                                         val_loader_with_viewable_transforms[i]["image"].to(device),
                                                         val_loader_with_viewable_transforms[i]["label"].to(device),
                                                         os.path.join(
                                                             EXPERIMENT_NAME,
                                                             "logs",
                                                             "segmentations",
                                                             "image_{}.png".format(i)
                                                         ))
                            for i in range(0, 3)]
                     ),
                     epoch_end_callback=call_many(
                          save_model_on_better_miou(os.path.join(EXPERIMENT_NAME, "saved/model.pt"),
                                                    0),
                          save_interesting_images(os.path.join(EXPERIMENT_NAME,
                                                               "logs",
                                                               "interesting",
                                                               "image.png"),
                                                  device)
                     ),
                     start_epoch=0)

Validation Batch:   0%|          | 0/3 [00:00<?, ?it/s]../train_semantic_segmentation.py:315: RuntimeWarning: invalid value encountered in true_divide
  return np.nanmean(np.diag(confusion_matrix) / union) if union.sum() > 0 else 0

Epoch:   0%|          | 0/50 [00:20<?, ?it/s]<00:00,  2.14s/it, loss=0.771, miou=0.00403]

Epoch 0, Validation loss: 0.5016014476617178, Validation mIoU: 0.006830125023331391


Epoch:   2%|▏         | 1/50 [00:49<24:19, 29.78s/it] 2.17s/it, loss=0.622, miou=0.0798]

Epoch 1, Validation loss: 0.39668790499369305, Validation mIoU: 0.1309586689439294
